In [1]:
from selenium import webdriver
import time

In [3]:
driver = webdriver.Chrome()

In [4]:
url = 'https://docs.google.com/forms/d/e/1FAIpQLScM3RBbQaf6vNa6ppDM5wrQgxMocEBlhh_6ez_WSaGJIzNbPw/viewform?pli=1'

In [5]:
driver.get(url)

In [13]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import random
import csv

class GoogleFormAutomator:
    def __init__(self, form_url):
        """Initialize the automator with the form URL."""
        self.form_url = form_url
        self.driver = None
    
    def setup_driver(self):
        """Set up the Chrome webdriver with appropriate options."""
        options = webdriver.ChromeOptions()
        # Uncomment the line below if you want to run in headless mode
        # options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option('excludeSwitches', ['enable-automation'])
        options.add_experimental_option('useAutomationExtension', False)
        
        self.driver = webdriver.Chrome(options=options)
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        self.driver.maximize_window()
        
    def open_form(self):
        """Open the Google Form."""
        self.driver.get(self.form_url)
        # Wait for form to load completely
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='listitem']"))
            )
            print("Form loaded successfully")
        except TimeoutException:
            print("Form loading timed out")
            self.driver.quit()
            return False
        return True
    
    def scroll_to_element(self, element):
        """Scroll element into view smoothly."""
        self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
        time.sleep(0.5)
        
    def select_radio_options(self, strategy="random"):
        """Select radio options for all radio sections."""
        # Find all question sections
        question_sections = self.driver.find_elements(By.CSS_SELECTOR, "div[role='listitem']")
        
        for i, section in enumerate(question_sections):
            # Check if this section contains radio buttons
            radio_options = section.find_elements(By.CSS_SELECTOR, "div[role='radio']")
            
            if not radio_options:
                # This might be a text section or other type of question
                continue
            
            # Select option based on strategy
            if strategy == "random":
                option_to_select = random.choice(radio_options)
            elif strategy == "first":
                option_to_select = radio_options[0]
            elif strategy == "last":
                option_to_select = radio_options[-1]
            else:
                option_to_select = radio_options[0]
            
            # Scroll to the option and use multiple click methods
            try:
                # Scroll the option into view
                self.scroll_to_element(option_to_select)
                
                # Try a few different methods to click
                try:
                    # Method 1: Direct click
                    option_to_select.click()
                except ElementClickInterceptedException:
                    try:
                        # Method 2: JavaScript click
                        self.driver.execute_script("arguments[0].click();", option_to_select)
                    except Exception:
                        # Method 3: Actions click
                        actions = ActionChains(self.driver)
                        actions.move_to_element(option_to_select).click().perform()
                
                print(f"Selected option in section {i+1}")
                # Small delay to prevent detection as bot
                time.sleep(random.uniform(0.7, 1.8))
            except Exception as e:
                print(f"Error selecting option in section {i+1}: {e}")
    
    def fill_text_area(self, text):
        """Fill the text area in the last section."""
        try:
            # Find all textareas on the page
            textareas = self.driver.find_elements(By.CSS_SELECTOR, "textarea")
            
            if not textareas:
                # Try finding by different selectors if direct textarea not found
                textareas = self.driver.find_elements(By.CSS_SELECTOR, "input[type='text']")
            
            if textareas:
                text_area = textareas[-1]  # Get the last textarea
                
                # Scroll to it
                self.scroll_to_element(text_area)
                
                # Clear any existing text and type new text
                text_area.clear()
                
                # Type with human-like delays
                for char in text:
                    text_area.send_keys(char)
                    time.sleep(random.uniform(0.03, 0.12))
                
                print("Text area filled successfully")
            else:
                print("No text area found")
        except Exception as e:
            print(f"Error filling text area: {e}")
    
    def submit_form(self):
        """Submit the form by clicking the submit button."""
        try:
            # Locate the submit button
            submit_buttons = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[role='button']"))
            )

            # Filter for the likely submit button (usually contains text like "Submit")
            for button in submit_buttons:
                button_text = button.text.lower()
                if any(word in button_text for word in ['සබිමිටි කරන්න', "submit", "send", "next", "submit form"]):
                    self.scroll_to_element(button)
                    button.click()
                    print(f"Clicked submit button with text: {button.text}")
                    break
            else:
                # If no submit button is found, try the next button
                next_buttons = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".freebirdFormviewerViewNavigationButtons .appsMaterialWizButtonPaperbuttonLabel"))
                )
                if next_buttons:
                    self.scroll_to_element(next_buttons[-1])
                    next_buttons[-1].click()
                    print("Clicked the next/submit button")
                else:
                    # If no buttons are found, press Enter
                    active_element = self.driver.switch_to.active_element
                    active_element.send_keys(Keys.ENTER)
                    print("Pressed Enter key to submit")

            # Wait for submission confirmation
            try:
                # Re-locate the confirmation element after submission
                WebDriverWait(self.driver, 20).until(
                    EC.any_of(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.freebirdFormviewerViewResponseConfirmationMessage")),
                        EC.url_contains("formResponse")
                    )
                )
                print("Form submitted successfully")
                return True
            except TimeoutException:
                print("Form submission confirmation not found")
                return False

        except Exception as e:
            print(f"Error submitting form: {e}")
            return False
    
    def close_driver(self):
        """Close the webdriver."""
        if self.driver:
            self.driver.quit()
            print("Driver closed")
    
    def automate_form(self, text_response, selection_strategy="random"):
        """Automate the entire form filling and submission process."""
        try:
            self.setup_driver()
            if not self.open_form():
                return False
            
            self.select_radio_options(strategy=selection_strategy)
            self.fill_text_area(text_response)
            
            # Add a small delay before submitting
            time.sleep(random.uniform(1, 2))
            
            result = self.submit_form()
            
            # Add a delay to see the confirmation
            time.sleep(3)
            
            self.close_driver()
            return result
            
        except Exception as e:
            print(f"Error during automation: {e}")
            self.close_driver()
            return False


# Example usage
if __name__ == "__main__":
    # Replace with your actual Google Form URL
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScM3RBbQaf6vNa6ppDM5wrQgxMocEBlhh_6ez_WSaGJIzNbPw/viewform?pli=1"
    
    # Read responses from a CSV file
    csv_file = "responses.csv"  # Replace with your CSV file path
    responses = []
    
    try:
        with open(csv_file, mode="r", encoding="utf-8") as file:
            reader = csv.reader(file)
            for row in reader:
                responses.append(row[0])  # Assuming each row has one response
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_file}' not found.")
        exit(1)
    
    automator = GoogleFormAutomator(form_url)
    
    # Run the automation for each response in the CSV file
    for i, response_text in enumerate(responses):
        print(f"\n--- Starting Form Submission {i+1} ---")
        print(f"Using response: {response_text}")
        
        success = automator.automate_form(response_text, selection_strategy="random")
        
        if success:
            print(f"Form submission {i+1} completed successfully")
        else:
            print(f"Form submission {i+1} failed")
        
        # Add a delay between submissions to avoid detection
        time.sleep(random.uniform(5, 10))  # Random delay between 5 and 10 seconds

# main automated algorithm 

In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import random
import csv

class GoogleFormAutomator:
    def __init__(self, form_url):
        """Initialize the automator with the form URL."""
        self.form_url = form_url
        self.driver = None
    
    def setup_driver(self):
        """Set up the Chrome webdriver with appropriate options."""
        options = webdriver.ChromeOptions()
        # Uncomment the line below if you want to run in headless mode
        # options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option('excludeSwitches', ['enable-automation'])
        options.add_experimental_option('useAutomationExtension', False)
        
        self.driver = webdriver.Chrome(options=options)
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        self.driver.maximize_window()
        
    def open_form(self):
        """Open the Google Form."""
        self.driver.get(self.form_url)
        # Wait for form to load completely
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='listitem']"))
            )
            print("Form loaded successfully")
        except TimeoutException:
            print("Form loading timed out")
            self.driver.quit()
            return False
        return True
    
    def scroll_to_element(self, element):
        """Scroll element into view smoothly."""
        self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
        time.sleep(0.5)
        
    def select_radio_options(self, strategy="random"):
        """Select radio options for all radio sections."""
        # Find all question sections
        question_sections = self.driver.find_elements(By.CSS_SELECTOR, "div[role='listitem']")
        
        for i, section in enumerate(question_sections):
            # Check if this section contains radio buttons
            radio_options = section.find_elements(By.CSS_SELECTOR, "div[role='radio']")
            
            if not radio_options:
                # This might be a text section or other type of question
                continue
            
            # Select option based on strategy
            if strategy == "random":
                option_to_select = random.choice(radio_options)
            elif strategy == "first":
                option_to_select = radio_options[0]
            elif strategy == "last":
                option_to_select = radio_options[-1]
            else:
                option_to_select = radio_options[0]
            
            # Scroll to the option and use multiple click methods
            try:
                # Scroll the option into view
                self.scroll_to_element(option_to_select)
                
                # Try a few different methods to click
                try:
                    # Method 1: Direct click
                    option_to_select.click()
                except ElementClickInterceptedException:
                    try:
                        # Method 2: JavaScript click
                        self.driver.execute_script("arguments[0].click();", option_to_select)
                    except Exception:
                        # Method 3: Actions click
                        actions = ActionChains(self.driver)
                        actions.move_to_element(option_to_select).click().perform()
                
                print(f"Selected option in section {i+1}")
                # Small delay to prevent detection as bot
                time.sleep(random.uniform(0.7, 1.8))
            except Exception as e:
                print(f"Error selecting option in section {i+1}: {e}")
    
    def fill_text_area(self, text):
        """Fill the text area in the last section."""
        try:
            # Find all textareas on the page
            textareas = self.driver.find_elements(By.CSS_SELECTOR, "textarea")
            
            if not textareas:
                # Try finding by different selectors if direct textarea not found
                textareas = self.driver.find_elements(By.CSS_SELECTOR, "input[type='text']")
            
            if textareas:
                text_area = textareas[-1]  # Get the last textarea
                
                # Scroll to it
                self.scroll_to_element(text_area)
                
                # Clear any existing text and type new text
                text_area.clear()
                
                # Type with human-like delays
                for char in text:
                    text_area.send_keys(char)
                    time.sleep(random.uniform(0.03, 0.12))
                
                print("Text area filled successfully")
            else:
                print("No text area found")
        except Exception as e:
            print(f"Error filling text area: {e}")
    
    def submit_form(self):
        """Submit the form by clicking the submit button."""
        try:
            # Locate the submit button
            submit_buttons = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[role='button']"))
            )

            # Filter for the likely submit button (usually contains text like "Submit")
            for button in submit_buttons:
                button_text = button.text.lower()
                if any(word in button_text for word in ['සබිමිටි කරන්න', "submit", "send", "next", "submit form"]):
                    self.scroll_to_element(button)
                    button.click()
                    print(f"Clicked submit button with text: {button.text}")
                    break
            else:
                # If no submit button is found, try the next button
                next_buttons = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".freebirdFormviewerViewNavigationButtons .appsMaterialWizButtonPaperbuttonLabel"))
                )
                if next_buttons:
                    self.scroll_to_element(next_buttons[-1])
                    next_buttons[-1].click()
                    print("Clicked the next/submit button")
                else:
                    # If no buttons are found, press Enter
                    active_element = self.driver.switch_to.active_element
                    active_element.send_keys(Keys.ENTER)
                    print("Pressed Enter key to submit")

            # Wait for submission confirmation
            try:
                # Re-locate the confirmation element after submission
                WebDriverWait(self.driver, 20).until(
                    EC.any_of(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.freebirdFormviewerViewResponseConfirmationMessage")),
                        EC.url_contains("formResponse")
                    )
                )
                print("Form submitted successfully")
                return True
            except TimeoutException:
                print("Form submission confirmation not found")
                return False

        except Exception as e:
            print(f"Error submitting form: {e}")
            return False
    
    def close_driver(self):
        """Close the webdriver."""
        if self.driver:
            self.driver.quit()
            print("Driver closed")
    
    def automate_form(self, text_response, selection_strategy="random"):
        """Automate the entire form filling and submission process."""
        try:
            self.setup_driver()
            if not self.open_form():
                return False
            
            self.select_radio_options(strategy=selection_strategy)
            self.fill_text_area(text_response)
            
            # Add a small delay before submitting
            time.sleep(random.uniform(1, 2))
            
            result = self.submit_form()
            
            # Add a delay to see the confirmation
            time.sleep(3)
            
            self.close_driver()
            return result
            
        except Exception as e:
            print(f"Error during automation: {e}")
            self.close_driver()
            return False


# Example usage
if __name__ == "__main__":
    # Replace with your actual Google Form URL
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScM3RBbQaf6vNa6ppDM5wrQgxMocEBlhh_6ez_WSaGJIzNbPw/viewform?pli=1"
    
    # Read responses from a CSV file
    csv_file = "responses.csv"  # Replace with your CSV file path
    responses = []
    
    try:
        with open(csv_file, mode="r", encoding="utf-8") as file:
            reader = csv.reader(file)
            for row in reader:
                responses.append(row[0])  # Assuming each row has one response
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_file}' not found.")
        exit(1)
    
    automator = GoogleFormAutomator(form_url)
    
    # Define how many submissions you want to run
    num_submissions = 5  # Change this value to control the number of submissions
    
    # Run the automation for the specified number of submissions
    for i in range(num_submissions):
        if i >= len(responses):
            print("No more responses in the CSV file.")
            break
        
        print(f"\n--- Starting Form Submission {i+1} ---")
        print(f"Using response: {responses[i]}")
        
        success = automator.automate_form(responses[i], selection_strategy="random")
        
        if success:
            print(f"Form submission {i+1} completed successfully")
        else:
            print(f"Form submission {i+1} failed")
        
        # Add a delay between submissions to avoid detection
        time.sleep(random.uniform(5, 10))  # Random delay between 5 and 10 seconds


--- Starting Form Submission 1 ---
Using response: I would love to see an option for online booking with real-time availability.
Form loaded successfully
Selected option in section 1
Selected option in section 2
Selected option in section 3
Selected option in section 4
Selected option in section 5
Selected option in section 6
Selected option in section 7
Selected option in section 8
Selected option in section 9
Selected option in section 10
Selected option in section 11
Text area filled successfully
Error submitting form: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.143); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001051ec2d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x00000001051e4934 cxxbridge1$str$ptr + 2708700
2   chromedriver               